In [1]:
import config
import json
import pandas as pd
import os

In [2]:
VERSION=config.VERSION
output_dir='../output_v%s/cskg' % VERSION
data_dir='../output_v%s' % VERSION

In [3]:
cn_nodes_file='%s/conceptnet/nodes_v%s.csv' % (data_dir, VERSION)
vg_nodes_file='%s/visualgenome/nodes_v%s.csv' % (data_dir, VERSION)
wn_nodes_file='%s/wordnet/nodes_v%s.csv' % (data_dir, VERSION)
wd_nodes_file='%s/wikidata/nodes_v%s.csv' % (data_dir, VERSION)

combined_nodes_file='%s/nodes_v%s.csv' % (output_dir, VERSION)

In [4]:
nodes_inputs=[cn_nodes_file,vg_nodes_file,wn_nodes_file,wd_nodes_file]

In [5]:
cn_edges_file='%s/conceptnet/edges_all_v%s.csv' % (data_dir, VERSION)
vg_edges_file='%s/visualgenome/edges_v%s.csv' % (data_dir, VERSION)
wn_edges_file='%s/wordnet/edges_v%s.csv' % (data_dir, VERSION)
wd_edges_file='%s/wikidata/edges_v%s.csv' % (data_dir, VERSION)

wn2wn_edges_file='%s/mappings/edges_v%s.csv' % (data_dir, VERSION)
wn2wd_edges_file='%s/mappings/edges_wn2wdt_v%s.csv' % (data_dir, VERSION)

combined_edges_file='%s/edges_v%s.csv' % (output_dir, VERSION)

In [6]:
edges_inputs=[cn_edges_file,
              vg_edges_file,
              wn_edges_file,wd_edges_file,wn2wn_edges_file,wn2wd_edges_file]

In [7]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Combine and store nodes

In [8]:
all_dfs=[]
for f in nodes_inputs:
    tmp_df=pd.read_csv(f, sep='\t', converters={5: eval})
    all_dfs.append(tmp_df)

/Users/filipilievski/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/filipilievski/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
combined_nodes = pd.concat(all_dfs)

In [10]:
l=combined_nodes[combined_nodes.duplicated(['id'], keep=False)]['id']

In [11]:
for e in l:
    if not e.startswith('wn:'): print(e)

vg:I2417082
vg:I2417082


In [12]:
len(combined_nodes)

18252226

In [13]:
# Drop duplicates
combined_nodes.drop_duplicates(subset ="id", 
                             keep = 'first', inplace = True)

In [14]:
len(combined_nodes)

18242590

In [15]:
nodes_in_nodes=set(combined_nodes.id.unique())

### Combine and store edges

In [16]:
all_dfs=[]
for f in edges_inputs:
    tmp_df=pd.read_csv(f, sep='\t', converters={5: eval})
    all_dfs.append(tmp_df)

In [17]:
combined_edges = pd.concat(all_dfs)

In [18]:
len(combined_edges)

38242649

In [19]:
l=combined_edges[combined_edges.duplicated(['subject', 'predicate','object'], keep=False)]

In [20]:
len(l)

24723

In [21]:
print(l['predicate'].unique())

['/r/RelatedTo' '/r/Antonym' '/r/DistinctFrom'
 '/r/EtymologicallyRelatedTo' '/r/Synonym' 'vg:InBoundingBox'
 'vg:PWordnetSynset' 'rdfs:subClassOf']


In [22]:
# Drop duplicates
combined_edges.drop_duplicates(subset =['subject', 'predicate','object'], 
                                 keep = 'first', inplace = True)

### Analysis and consistency

In [23]:
uniq_subjects=combined_edges.subject.unique()
uniq_objects=combined_edges.object.unique()
nodes_in_edges=set(uniq_subjects) | set(uniq_objects)

In [24]:
len(nodes_in_edges)

18305970

In [25]:
len(nodes_in_nodes-nodes_in_edges)

468

In [26]:
#nodes_in_nodes-nodes_in_edges

In [27]:
missing_nodes=nodes_in_edges-nodes_in_nodes

In [28]:
len(missing_nodes)

63848

In [29]:
rows=[]
for m in missing_nodes:
    if m.startswith('wd:'):
        datasource=config.wdt_ds
    elif m.startswith('wn:'):
        datasource=config.wn_ds
    elif m.startswith('/c/en'):
        datasource=config.cn_ds
    a_row=[m, "", "", "", datasource, {}]
    rows.append(a_row)

In [32]:
new_df=pd.DataFrame(rows, columns=config.nodes_cols)
combined_nodes_plus = pd.concat([combined_nodes, new_df])

In [33]:
len(combined_nodes_plus)

18306438

In [34]:
len(combined_edges)

38229993

### Store data

In [35]:
combined_nodes_plus.sort_values('id').to_csv(combined_nodes_file, index=False, sep='\t')

In [36]:
combined_edges.sort_values(by=['subject', 'predicate','object']).to_csv(combined_edges_file, index=False, sep='\t')